# Setup

In [46]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np

In [47]:
sys.path.append(os.path.abspath("../../.."))

In [48]:
from src.experiment.helpers.variables import dataset_root_dir
from src.experiment.helpers.utils import get_sorted_class_percentages_label_encoded

In [49]:
NAME = "nyt-comments-april17"
CLASSIFICATION_TYPE = "multiclass"

In [50]:
DATASET_DIR = dataset_root_dir / CLASSIFICATION_TYPE / NAME
print(DATASET_DIR)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multiclass\nyt-comments-april17


In [51]:
balanced_name = NAME + "_balanced"
DATASET_DIR_BALANCED = dataset_root_dir / CLASSIFICATION_TYPE / balanced_name
print(DATASET_DIR_BALANCED)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multiclass\nyt-comments-april17_balanced


In [52]:
imbalanced_name = NAME + "_imbalanced"
DATASET_DIR_IMBALANCED = dataset_root_dir / CLASSIFICATION_TYPE / imbalanced_name
print(DATASET_DIR_IMBALANCED)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multiclass\nyt-comments-april17_imbalanced


# Dataset loading

In [88]:
X = pd.read_csv(DATASET_DIR / 'X.csv')
y = pd.read_csv(DATASET_DIR / 'y.csv')

In [89]:
X

,00,000,01,02,03,04,05,06,07,08,...,advisor,advisors,advocacy,advocate,advocated,advocates,advocating,aetna,afar,affair
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260965,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
y

,typeOfMaterial
0,0
1,0
2,0
3,1
4,1
...,...
260962,4
260963,4
260964,4
260965,4


# Cut classes to match balanced dataset

In [11]:
y_balanced = pd.read_csv(DATASET_DIR_BALANCED / 'y_2.csv')
y_balanced

,typeOfMaterial
0,3
1,3
2,3
3,3
4,3
...,...
7828,4
7829,4
7830,4
7831,4


In [14]:
# Get the set of classes present in y_balanced
classes_in_balanced = set(y_balanced['typeOfMaterial'].unique())
print(f"Classes in balanced dataset: {classes_in_balanced}")

# Create mask for samples in y that are present in y_balanced
mask = y['typeOfMaterial'].isin(classes_in_balanced)

# # Filter X and y
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

Classes in balanced dataset: {np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(8)}


# Reduce class indexes by 1

In [11]:
#check if any label "0" is present in y
if (y == 0).any().any():
    print("Label '0' is present in y")
else:
    print("Label '0' is not present in y")

Label '0' is not present in y


In [13]:
# reduce y by 1
y['Class'] = y['Class'] - 1

# Remove gaps in labels

In [96]:
# remove gaps in labels, as in if there is a label "0" and then a label "2", turn the 2 into a 1
y = y.apply(lambda x: pd.factorize(x)[0])

# Dataset info

In [16]:
# number of samples
len(X)

259817

In [17]:
# number of classes (number of unique numbers in y)
len(np.unique(y.values))

6

In [18]:
# Convert y to int if needed
class_counts = y.value_counts()
print(class_counts)

typeOfMaterial
2                 137004
1                  96229
0                  21281
3                   2243
4                   1840
5                   1220
Name: count, dtype: int64


In [19]:
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
print(class_percentages_sorted)

2    52.730961
1    37.037222
0     8.190765
3     0.863300
4     0.708191
5     0.469561
Name: proportion, dtype: float64


# Functions

In [20]:
def get_least_represented_class_names(class_percentages_sorted, percent):
    num_classes = len(class_percentages_sorted)
    bottom_percent_count = int(np.ceil(num_classes * percent))
    least_represented_classes = class_percentages_sorted.tail(bottom_percent_count).index.tolist()
    return least_represented_classes

In [21]:
def get_most_represented_class_names(class_percentages_sorted, percent):
    num_classes = len(class_percentages_sorted)
    top_percent_count = int(np.ceil(num_classes * percent))
    most_represented_classes = class_percentages_sorted.head(top_percent_count).index.tolist()
    return most_represented_classes

In [22]:
def show_summary(X, y):
    print("Number of samples:", len(X))
    print("Number of classes:", len(np.unique(y.values)))

    class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
    
    highest_class_percentage = class_percentages_sorted.iloc[0]
    lowest_class_percentage = class_percentages_sorted.iloc[-1]
    
    if lowest_class_percentage != 0:
        ratio = highest_class_percentage / lowest_class_percentage
        print(f"Times higher: {round(ratio, 2)}")
    else:
        ratio = np.nan
        print("The lowest class percentage is 0, cannot calculate ratio.")
        
    average_class_percentage = class_percentages_sorted.mean()
    class_percentages_std = class_percentages_sorted.std()

    cov = class_percentages_std / average_class_percentage
    print(f"CoV: {round(cov, 3)}")
    
    print(class_percentages_sorted)
    
    return class_percentages_sorted

In [23]:
def remove_class(X, y, chosen_class_index):
    # If DataFrame, select the first column
    if isinstance(y, pd.DataFrame):
        y_col = y.columns[0]
        y_series = y[y_col]
    else:
        y_series = y


    print(f"Removing class: {chosen_class_index}")
    
    # Create mask from 1D series
    mask = y_series != chosen_class_index

    # Filter X and y using 1D mask
    return X.loc[mask], y.loc[mask]


In [24]:
# removes SAMPLES
def remove_few_samples_of_class(X, y, class_index, step):
    indices_to_remove = y[y.values.flatten() == class_index].index[:step]
    X_trimmed = X.drop(indices_to_remove)
    y_trimmed = y.drop(indices_to_remove)
    
    return X_trimmed, y_trimmed

In [25]:
def remove_samples_from_all_classes(X, y, step):
    class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
    for chosen_class in class_percentages_sorted.index:
        X, y = remove_few_samples_of_class(X, y, chosen_class, step=step)   
    return X, y

# Dataset trimming

In [23]:
some_classes = get_most_represented_class_names(class_percentages_sorted, 0.8)
some_classes

[1, 0]

In [ ]:
# remove top class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[0]

X,y = remove_class(X, y, chosen_class)
class_percentages_sorted = show_summary(X, y)

In [95]:
# remove bottom class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[-1]

X,y = remove_class(X, y, chosen_class)
class_percentages_sorted = show_summary(X, y)

Removing class: 10
Number of samples: 259817
Number of classes: 6
Times higher: 112.3
CoV: 1.356
2    52.730961
1    37.037222
0     8.190765
3     0.863300
4     0.708191
8     0.469561
Name: proportion, dtype: float64


In [201]:
# remove samples from top class

class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[1]
X,y = remove_few_samples_of_class(X, y, chosen_class, step=300)

class_percentages_sorted = show_summary(X, y)

Number of samples: 19817
Number of classes: 6
Times higher: 6.97
CoV: 0.802
2    42.912651
1    16.798708
0    13.528788
3    11.318565
4     9.284957
5     6.156330
Name: proportion, dtype: float64


In [1095]:
# remove samples from bottom class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[-6]

X,y = remove_few_samples_of_class(X, y, chosen_class, step=10)
class_percentages_sorted = show_summary(X, y)

Number of samples: 7866
Number of classes: 6
Times higher: 6.75
CoV: 0.616
2    36.028477
1    16.259853
4    15.522502
3    14.251208
0    12.598525
5     5.339436
Name: proportion, dtype: float64


In [344]:
X,y = remove_samples_from_all_classes(X, y, step=200)
class_percentages_sorted = show_summary(X, y)

Number of samples: 14561
Number of classes: 2
Times higher: 4.16
CoV: 0.866
0    80.605728
1    19.394272
Name: proportion, dtype: float64


# Trimmed dataset - info

In [1096]:
class_percentages_sorted = show_summary(X, y)

Number of samples: 7866
Number of classes: 6
Times higher: 6.75
CoV: 0.616
2    36.028477
1    16.259853
4    15.522502
3    14.251208
0    12.598525
5     5.339436
Name: proportion, dtype: float64


# Save

### Basic

In [1847]:
X.to_csv(DATASET_DIR / "X.csv", index=False)
y.to_csv(DATASET_DIR / "y.csv", index=False)

In [1848]:
class_percentages_sorted.to_csv(DATASET_DIR / "class_percentages.csv", header=False)

### Balanced

In [454]:
DATASET_DIR_BALANCED.mkdir(parents=True, exist_ok=True)
X.to_csv(DATASET_DIR_BALANCED / "X.csv", index=False)
y.to_csv(DATASET_DIR_BALANCED / "y.csv", index=False)

In [455]:
class_percentages_sorted.to_csv(DATASET_DIR_BALANCED / "class_percentages.csv", header=False)

### Imbalanced

In [1097]:
DATASET_DIR_IMBALANCED.mkdir(parents=True, exist_ok=True)
X.to_csv(DATASET_DIR_IMBALANCED / "X.csv", index=False)
y.to_csv(DATASET_DIR_IMBALANCED / "y.csv", index=False)

In [1098]:
class_percentages_sorted.to_csv(DATASET_DIR_IMBALANCED / "class_percentages.csv", header=False)